In [ ]:
from model import run_exp
import utils as utl

# computational set up

In [ ]:
# set up environment
path_comp = './2023_03_23_completeness_630_final.csv'
path_con = './2023_03_23_connectivity_630_final.parquet'
path_res = './results/example/'

config = {
    'path_comp'         : path_comp, 
    'path_con'          : path_con, 
    'path_res'          : path_res,     # store results here
    'n_proc'            : 4,            # number of CPUs to use, -1 uses all available CPU
}

In [ ]:
# custon neuron names
path_name = './flywire_ids_630.xlsx'

# sheet names of the xls file to include
sheets_pair = [ # pair of neurons (left+right per row)
    'stop',
    'walk',
    'walk_outputs',
    ]

sheets_single = [ # single neurons (one per row)
    # 'sugar', 
    # 'ovidn', 
    # 'bitter', 
    ]

name2flyid = utl.create_name_dict(path_name, path_comp, sheets_pair, sheets_single)

In [ ]:
# lists of neuron groups
l_p9 = ['P9_l', 'P9_r']
l_bb = ['BB_r', 'BB_l']
l_cdn = ['P9-cDN1_l', 'P9-cDN1_r']

# run experiments

In [ ]:
# P9 activation
instructions = [ 
    (0, 'stim', l_p9), 
    (1, 'end', [])
    ]

run_exp(exp_name='P9', exp_inst=instructions, name2flyid=name2flyid, **config, force_overwrite=True)

In [ ]:
# more complex instuctions: 
# (i) activate P9 
# (ii) after some delay, activate BB 
# (iii) after some more delay, silence BB

# caveate
# - silenced neurons cannot be unsilenced
# - silenced neurons will still spike, if they also have a Poisson input (stim)

instructions = [ 
    (0, 'stim', l_p9), 
    (0.25, 'stim', l_bb),
    (0.75, 'slnc', l_bb),
    (1, 'end', []),
    ]
run_exp(exp_name='P9+BB_slnc', exp_inst=instructions, name2flyid=name2flyid, **config, force_overwrite=True)

In [ ]:
# changing model parameters
from model import default_params as params
from brian2 import Hz

instructions = [ 
    (0, 'stim', l_p9), 
    (1, 'end', [])
    ]

params['r_poi'] = 250 * Hz

run_exp(exp_name='P9_ultra', exp_inst=instructions, name2flyid=name2flyid, **config, params=params, force_overwrite=True)

In [ ]:
# use 2 different stimulation frequencies
# frequency for stim2 is controlled via params['r_poi2']

instructions = [ 
    (0, 'stim', l_p9), 
    (0, 'stim2', l_bb),
    (1, 'end', []),
    ]

run_exp(exp_name='P9+BB2', exp_inst=instructions, name2flyid=name2flyid, **config, force_overwrite=True)

# Process results

In [ ]:
# choose experiments to load
outputs = [
    './results/example/P9+BB_slnc.parquet',
    './results/example/P9_ultra.parquet',
]

# load spike times, calculate rate + standard deviation
df_spkt = utl.load_exps(outputs)
df_rate, df_std = utl.get_rate(df_spkt, duration=1)

# convert IDs to custom names, if possible
df_rate = utl.rename_index(df_rate, name2flyid)
df_std = utl.rename_index(df_std, name2flyid)

# save as spreadsheets
utl.save_xls(df_rate, './results/example/rate.xlsx')
utl.save_xls(df_std, './results/example/rate_std.xlsx')

In [ ]:
# raster plots
neu = l_p9 + l_bb + l_cdn
utl.plot_raster(df_spkt, neu, name2flyid=name2flyid, xlims=(0, 1))

In [ ]:
# firing rates
utl.plot_rate(df_spkt, neu, xlims=(0, 1), name2flyid=name2flyid)

In [ ]:
# select top 20 neurons
top20 = df_rate.sort_values(by='P9_ultra', ascending=False).head(20)
top20

In [ ]:
# plot heatmap
utl.plot_rate_heatmap(df_spkt, top20.index, xlims=(0, 1), name2flyid=name2flyid, do_zscore=False)

# Graph representation

In [ ]:
# load connectome into graph
G = utl.get_full_graph(path_comp, path_con)
print(len(G.nodes))

In [ ]:
# select subgraph based on simulation results
output = './results/example/P9.parquet'
utl.write_graph(G, output, name2flyid=name2flyid)